<a href="https://colab.research.google.com/github/nimrashaheen001/Programming_for_AI/blob/main/thesisfusion1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os

zip_path = '/content/Data-1.zip'
extract_path = '/content/dataset'

if zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction complete.")
else:
    print("❌ The file is not a valid zip file.")

Extraction complete.


In [3]:
import pandas as pd
import glob, os

# 1) Load tabular data
tab_lookup = pd.read_csv("/content/tabular.csv")

# 2) Normalize ImageID column
tab_lookup['ImageID'] = tab_lookup['ImageID'].astype(str)

# If your CSV has no extension, but your dataset has .png
tab_lookup['ImageID'] = tab_lookup['ImageID'].apply(lambda x: x if x.endswith(".png") else x + ".png")

# If CSV has .jpg but dataset has .png
# tab_lookup['ImageID'] = tab_lookup['ImageID'].str.replace(".jpg", ".png")

# 3) Check matching
img_files = [os.path.basename(f) for f in glob.glob("/content/images/*/*")]  # adjust to your path
img_set = set(img_files)
tab_set = set(tab_lookup['ImageID'])

print("CSV sample:", list(tab_set)[:5])
print("Image sample:", list(img_set)[:5])
print("Overlap:", len(img_set & tab_set))


CSV sample: ['OAS1_0031_MR1_mpr-1_147.jpg.png', 'OAS1_0052_MR1_mpr-1_106.jpg.png', 'OAS1_0021_MR1_mpr-3_157.jpg.png', 'OAS1_0001_MR1_mpr-3_112.jpg.png', 'OAS1_0028_MR1_mpr-2_123.jpg.png']
Image sample: []
Overlap: 0


In [4]:
import glob

# Recursively search for images under /content
all_imgs = glob.glob("/content/**/*.png", recursive=True) + glob.glob("/content/**/*.jpg", recursive=True)

print("Total images found:", len(all_imgs))
print("Sample paths:", all_imgs[:5])

Total images found: 2982
Sample paths: ['/content/dataset/Data/Data/AD/OAS1_0003_MR1_mpr-1_128.jpg', '/content/dataset/Data/Data/AD/OAS1_0021_MR1_mpr-3_117.jpg', '/content/dataset/Data/Data/AD/OAS1_0003_MR1_mpr-1_156.jpg', '/content/dataset/Data/Data/AD/OAS1_0022_MR1_mpr-1_156.jpg', '/content/dataset/Data/Data/AD/OAS1_0015_MR1_mpr-1_112.jpg']


In [5]:
import pandas as pd
import os
import glob

# === 1. Load your CSV ===
csv_path = "/content/tabular.csv"   # change if different
tab_lookup = pd.read_csv(csv_path)

print("[Before] Shape:", tab_lookup.shape)
print("[Before] Sample IDs:", tab_lookup['ImageID'].head().tolist())

# === 2. Clean ImageID column ===
# remove double extensions like ".jpg.png"
tab_lookup['ImageID'] = tab_lookup['ImageID'].str.replace(".jpg.png", ".jpg", regex=False)

# normalize to lowercase (in case your files are lowercase)
tab_lookup['ImageID'] = tab_lookup['ImageID'].str.lower()

print("[After] Sample IDs:", tab_lookup['ImageID'].head().tolist())

# === 3. Collect real image filenames ===
all_imgs = glob.glob("/content/dataset/Data/Data/**/*.jpg", recursive=True)
img_files = [os.path.basename(f).lower() for f in all_imgs]

print("Total images found:", len(img_files))
print("Sample real files:", img_files[:5])

# === 4. Verify overlap ===
tab_set = set(tab_lookup['ImageID'])
img_set = set(img_files)
overlap = tab_set & img_set

print(f"[Check] CSV IDs: {len(tab_set)}, Image files: {len(img_set)}, Overlap: {len(overlap)}")

if len(overlap) == 0:
    print("❌ Still no overlap! Check extensions or path issues.")
else:
    print("✅ Overlap found:", len(overlap))

# === 5. Save fixed CSV ===
fixed_path = "/content/tabular_fixed.csv"
tab_lookup.to_csv(fixed_path, index=False)
print(f"✅ Fixed CSV saved to {fixed_path}")


[Before] Shape: (2982, 9)
[Before] Sample IDs: ['OAS1_0003_MR1_mpr-1_100.jpg', 'OAS1_0003_MR1_mpr-1_101.jpg', 'OAS1_0003_MR1_mpr-1_102.jpg', 'OAS1_0003_MR1_mpr-1_103.jpg', 'OAS1_0003_MR1_mpr-1_104.jpg']
[After] Sample IDs: ['oas1_0003_mr1_mpr-1_100.jpg', 'oas1_0003_mr1_mpr-1_101.jpg', 'oas1_0003_mr1_mpr-1_102.jpg', 'oas1_0003_mr1_mpr-1_103.jpg', 'oas1_0003_mr1_mpr-1_104.jpg']
Total images found: 2982
Sample real files: ['oas1_0003_mr1_mpr-1_128.jpg', 'oas1_0021_mr1_mpr-3_117.jpg', 'oas1_0003_mr1_mpr-1_156.jpg', 'oas1_0022_mr1_mpr-1_156.jpg', 'oas1_0015_mr1_mpr-1_112.jpg']
[Check] CSV IDs: 2982, Image files: 2982, Overlap: 2982
✅ Overlap found: 2982
✅ Fixed CSV saved to /content/tabular_fixed.csv


In [6]:
tab_lookup = pd.read_csv("/content/tabular_fixed.csv")


In [9]:
# ================================
# Multimodal TransUNet + TabNet Top-K Fusion (Robust ID/Class Fix)
# ================================
# - Auto-fixes CSV ↔ image filename mismatches (case, extensions, double extensions)
# - Auto-maps class label aliases (e.g., MCI <-> MCI-1)
# - Concatenates TransUNet CLS embeddings + top-k tabular features
# - Early stopping + test metrics

import os
import re
import random
from pathlib import Path
from typing import Dict, Optional, List, Tuple

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score

# ----------------
# Config
# ----------------
class Cfg:
    # Paths
    data_root = "/content/dataset/Data/Data"
    tabular_csv = "/content/tabular_fixed.csv"           # full tabular file
    topk_csv    = "/content/tabular_top10_features.csv"  # optional (from TabNet)
    id_col = "ImageID"
    target_col = "Class"   # your CSV uses "Class"
    # Image
    img_size = 224
    in_channels = 3
    # Train
    batch_size = 16
    num_workers = 2
    seed = 42
    epochs = 100
    patience = 6
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Optim
    lr_head = 1e-4
    lr_backbone = 1e-5
    weight_decay = 1e-4
    # TransUNet-ish
    patch_size = 16
    embed_dim = 768
    num_heads = 8
    num_layers = 4
    freeze_backbone = True
    unfreeze_epoch = 5

cfg = Cfg()

# Repro
def set_seed(s):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed(cfg.seed)

# ----------------
# Filename & class normalization helpers
# ----------------
def normalize_ext_to_jpg(name: str) -> str:
    """
    Normalize a filename to .jpg and lowercase.
    Also fixes double extensions like `.jpg.png` -> `.jpg`, `.png.jpg` -> `.jpg`.
    """
    n = name.strip().lower()
    # remove any path parts, keep basename only
    n = os.path.basename(n)
    # strip double extensions
    n = n.replace(".jpg.png", ".jpg").replace(".png.jpg", ".jpg")
    # if it has an extension other than jpg/png/jpeg, keep it; otherwise standardize to .jpg
    root, ext = os.path.splitext(n)
    if ext in [".jpeg", ".png", ".jpg"]:
        n = root + ".jpg"
    return n

def stem_without_ext(name: str) -> str:
    """Lowercased basename stem (no extension)."""
    return os.path.splitext(os.path.basename(name.strip().lower()))[0]

def class_alias(name: str) -> str:
    """
    Normalize class labels across CSV and folders.
    Adjust here if your CSV/folders use slightly different labels.
    """
    if name is None:
        return ""
    s = str(name).strip()
    # Common aliases:
    # Some datasets use "MCI" in CSV and "MCI-1" in folders, or vice versa.
    if s.upper() == "MCI":
        return "MCI-1"
    if s.upper() == "MCI-1":
        return "MCI-1"
    if s.upper() == "AD":
        return "AD"
    if s.upper() == "CN":
        return "CN"
    # Fallback: return as-is
    return s

# ----------------
# Load tabular (prefer top-k)
# ----------------
if os.path.exists(cfg.topk_csv):
    df_tab = pd.read_csv(cfg.topk_csv)
    print(f"[Info] Loaded top-k features from {cfg.topk_csv} -> shape {df_tab.shape}")
else:
    df_tab = pd.read_csv(cfg.tabular_csv)
    print(f"[Warn] {cfg.topk_csv} not found. Falling back to {cfg.tabular_csv} -> shape {df_tab.shape}")

# Basic checks
if cfg.id_col not in df_tab.columns or cfg.target_col not in df_tab.columns:
    raise ValueError(f"CSV must contain '{cfg.id_col}' and '{cfg.target_col}'.")

# Drop columns that shouldn't be features if present
drop_cols = {cfg.id_col, cfg.target_col}
# Drop Sex only if it exists
if "Sex" in df_tab.columns:
    drop_cols.add("Sex")

feature_cols = [c for c in df_tab.columns if c not in drop_cols]
print(f"[Info] Using {len(feature_cols)} tabular features: {feature_cols[:8]}{' ...' if len(feature_cols)>8 else ''}")

# Normalize CSV IDs & classes; build lookup dict with *multiple keys* per row
tab_dict: Dict[str, Tuple[np.ndarray, str]] = {}
no_ext_rows = 0
for _, row in df_tab.iterrows():
    raw_id = str(row[cfg.id_col])
    raw_cls = class_alias(row[cfg.target_col])
    feats = row[feature_cols].to_numpy(dtype=np.float32)

    # Prepare multiple keys for robust matching
    keys = set()

    # 1) as normalized .jpg
    keys.add(normalize_ext_to_jpg(raw_id))

    # 2) if CSV gave a stem (no extension), add both .jpg and .png variants
    root, ext = os.path.splitext(raw_id)
    if ext == "":
        no_ext_rows += 1
        keys.add((root.strip().lower() + ".jpg"))
        keys.add((root.strip().lower() + ".png"))

    # 3) also allow the pure stem (for safety, we'll only compare against stems if needed)
    keys.add(stem_without_ext(raw_id))

    for k in keys:
        tab_dict[k] = (feats, raw_cls)

if no_ext_rows:
    print(f"[Info] CSV rows without extension: {no_ext_rows}")

# ----------------
# Image transforms
# ----------------
def to_three_channels(img_tensor: torch.Tensor) -> torch.Tensor:
    """If tensor is (1,H,W) replicate to (3,H,W)."""
    if img_tensor.ndim == 3 and img_tensor.size(0) == 1:
        return img_tensor.repeat(3, 1, 1)
    return img_tensor

transform_train = transforms.Compose([
    transforms.Resize((cfg.img_size, cfg.img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(25),
    transforms.RandomResizedCrop(cfg.img_size, scale=(0.85, 1.0)),
    transforms.ToTensor(),
    transforms.Lambda(to_three_channels),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

transform_eval = transforms.Compose([
    transforms.Resize((cfg.img_size, cfg.img_size)),
    transforms.ToTensor(),
    transforms.Lambda(to_three_channels),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

# ----------------
# Dataset: robust join by filename (with and without ext) + class aliasing
# ----------------
class MultiModalDataset(Dataset):
    def __init__(self, image_folder: ImageFolder, tab_dict: Dict[str, Tuple[np.ndarray, str]],
                 feature_cols: List[str], is_train: bool, debug_limit: int = 10):
        super().__init__()
        self.ifolder = image_folder
        self.tab_dict = tab_dict
        self.feature_cols = feature_cols

        kept, miss_tab, cls_mismatch = 0, 0, 0
        missing_examples, mismatch_examples = [], []

        filtered_samples: List[Tuple[str, int]] = []
        for path, y in self.ifolder.samples:
            base = os.path.basename(path)
            base_norm = normalize_ext_to_jpg(base)          # e.g., abc.jpg (lower)
            base_stem = stem_without_ext(base_norm)         # e.g., abc
            folder_cls = class_alias(self.ifolder.classes[y])

            row = None
            # Try matching by normalized filename with extension
            if base_norm in self.tab_dict:
                row = self.tab_dict[base_norm]
            # Else try by original lower filename (in case extension normalized differently in CSV)
            elif base.lower() in self.tab_dict:
                row = self.tab_dict[base.lower()]
            # Else try by stem (CSV may have stem key)
            elif base_stem in self.tab_dict:
                row = self.tab_dict[base_stem]
            # Else try also .png variant in case images were .png but normalized
            else:
                alt_png = base_stem + ".png"
                if alt_png in self.tab_dict:
                    row = self.tab_dict[alt_png]

            if row is None:
                miss_tab += 1
                if len(missing_examples) < debug_limit:
                    missing_examples.append(base)
                continue

            feats, csv_cls = row
            csv_cls_alias = class_alias(csv_cls)

            if csv_cls_alias != folder_cls:
                cls_mismatch += 1
                if len(mismatch_examples) < debug_limit:
                    mismatch_examples.append((base, csv_cls, folder_cls))
                continue

            filtered_samples.append((path, y))
            kept += 1

        self.ifolder.samples = filtered_samples
        self.ifolder.targets = [s[1] for s in filtered_samples]

        mode = "train" if is_train else "eval"
        print(f"[{mode}] Kept {len(self.ifolder)} images | Missing tab: {miss_tab} | Class mismatch: {cls_mismatch}")
        if miss_tab > 0:
            print(f"[{mode}] Example missing IDs (up to {debug_limit}): {missing_examples}")
        if cls_mismatch > 0:
            print(f"[{mode}] Example class mismatches (up to {debug_limit}): {mismatch_examples}")

    def __len__(self):
        return len(self.ifolder)

    def __getitem__(self, idx):
        path, y = self.ifolder.samples[idx]
        img = self.ifolder.loader(path)
        if not isinstance(img, Image.Image):
            img = Image.open(path).convert("RGB")
        img = self.ifolder.transform(img)

        base = os.path.basename(path)
        base_norm = normalize_ext_to_jpg(base)
        base_stem = stem_without_ext(base_norm)

        # Re-resolve the features the same way as in __init__
        row = ( self.tab_dict.get(base_norm)
                or self.tab_dict.get(base.lower())
                or self.tab_dict.get(base_stem)
                or self.tab_dict.get(base_stem + ".png") )
        feats_np, _ = row
        feats = torch.from_numpy(feats_np)  # (F,)

        return img, feats, y, Path(path).name

# ----------------
# ImageFolder + splits
# ----------------
base_folder = ImageFolder(root=cfg.data_root, transform=transform_train)
num_classes = len(base_folder.classes)
print(f"[Info] Found {len(base_folder)} images across {num_classes} classes: {base_folder.classes}")

# Same files; separate instances for per-split transforms
folder_train = ImageFolder(root=cfg.data_root, transform=transform_train)
folder_val   = ImageFolder(root=cfg.data_root, transform=transform_eval)
folder_test  = ImageFolder(root=cfg.data_root, transform=transform_eval)

# Split (random split across all images)
all_indices = list(range(len(base_folder)))
random.shuffle(all_indices)
n_total = len(all_indices)
n_train = int(0.7 * n_total)
n_val   = int(0.15 * n_total)
n_test  = n_total - n_train - n_val

idx_train = all_indices[:n_train]
idx_val   = all_indices[n_train:n_train+n_val]
idx_test  = all_indices[n_train+n_val:]

def subset_folder(in_folder: ImageFolder, keep_indices: List[int]) -> ImageFolder:
    new = ImageFolder(root=in_folder.root, transform=in_folder.transform, loader=in_folder.loader)
    new.classes = in_folder.classes
    new.class_to_idx = in_folder.class_to_idx
    new.samples = [in_folder.samples[i] for i in keep_indices]
    new.targets = [s[1] for s in new.samples]
    return new

folder_train = subset_folder(folder_train, idx_train)
folder_val   = subset_folder(folder_val, idx_val)
folder_test  = subset_folder(folder_test, idx_test)

# Wrap with MultiModalDataset
ds_train = MultiModalDataset(folder_train, tab_dict, feature_cols, is_train=True)
ds_val   = MultiModalDataset(folder_val,   tab_dict, feature_cols, is_train=False)
ds_test  = MultiModalDataset(folder_test,  tab_dict, feature_cols, is_train=False)

# Safety check to avoid DataLoader on empty datasets
def ensure_nonempty(ds, name):
    if len(ds) == 0:
        raise RuntimeError(
            f"{name} dataset is empty after matching.\n"
            f"→ Check the printed 'Missing tab' examples above and adjust normalization rules in class_alias()/normalize_ext_to_jpg()."
        )
ensure_nonempty(ds_train, "Train")
ensure_nonempty(ds_val, "Val")
ensure_nonempty(ds_test, "Test")

train_loader = DataLoader(ds_train, batch_size=cfg.batch_size, shuffle=True,  num_workers=cfg.num_workers)
val_loader   = DataLoader(ds_val,   batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)
test_loader  = DataLoader(ds_test,  batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

# ----------------
# TransUNet-ish backbone -> CLS embeddings
# ----------------
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        num_patches = (img_size // patch_size) * (img_size // patch_size)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.proj(x)                     # B, E, H/P, W/P
        x = x.flatten(2).transpose(1, 2)     # B, N, E
        cls = self.cls_token.expand(x.size(0), -1, -1)
        x = torch.cat((cls, x), dim=1)       # B, N+1, E
        x = x + self.pos_embed
        return self.dropout(x)

class TransUNetEncoder(nn.Module):
    """Transformer encoder returning normalized CLS embedding."""
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768, num_heads=8, num_layers=4):
        super().__init__()
        self.patch = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        enc_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads,
                                               dim_feedforward=embed_dim*4, dropout=0.1,
                                               batch_first=False)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = self.patch(x)         # B, N+1, E
        x = x.permute(1,0,2)      # S, B, E
        x = self.encoder(x)       # S, B, E
        cls = x[0]                # B, E
        return self.norm(cls)     # B, E

# ----------------
# Fusion head
# ----------------
class FusionHead(nn.Module):
    def __init__(self, img_dim: int, tab_dim: int, num_classes: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(img_dim + tab_dim, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(32, num_classes)
        )

    def forward(self, img_feat, tab_feat):
        x = torch.cat([img_feat, tab_feat], dim=1)
        return self.net(x)

# ----------------
# Build models
# ----------------
img_encoder = TransUNetEncoder(
    img_size=cfg.img_size, patch_size=cfg.patch_size, in_channels=cfg.in_channels,
    embed_dim=cfg.embed_dim, num_heads=cfg.num_heads, num_layers=cfg.num_layers
).to(cfg.device)

fusion_head = FusionHead(img_dim=cfg.embed_dim, tab_dim=len(feature_cols), num_classes=len(base_folder.classes)).to(cfg.device)

# Freeze backbone initially (optional)
def set_freeze(model: nn.Module, freeze: bool):
    for p in model.parameters():
        p.requires_grad = not freeze

set_freeze(img_encoder, cfg.freeze_backbone)

# Optimizer with two groups
optim_params = [
    {"params": fusion_head.parameters(), "lr": cfg.lr_head},
    {"params": [p for p in img_encoder.parameters() if p.requires_grad], "lr": cfg.lr_backbone},
]
optimizer = optim.AdamW(optim_params, weight_decay=cfg.weight_decay)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# ----------------
# Train / Eval
# ----------------
def run_epoch(loader, train: bool, epoch: int, desc: str):
    (img_encoder.train() if train else img_encoder.eval())
    (fusion_head.train() if train else fusion_head.eval())

    total_loss, preds_all, labels_all = 0.0, [], []
    loop = tqdm(loader, desc=desc, leave=False)

    for imgs, tabs, ys, _names in loop:
        imgs = imgs.to(cfg.device)
        tabs = tabs.to(cfg.device).float()
        ys   = ys.to(cfg.device).long()

        with torch.set_grad_enabled(train):
            img_feats = img_encoder(imgs)
            logits = fusion_head(img_feats, tabs)
            loss = criterion(logits, ys)

            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        preds_all.extend(logits.argmax(1).detach().cpu().numpy())
        labels_all.extend(ys.detach().cpu().numpy())

    epoch_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(labels_all, preds_all)
    f1  = f1_score(labels_all, preds_all, average="macro")
    return epoch_loss, acc, f1

best_val_acc = 0.0
wait = 0

for epoch in range(cfg.epochs):
    # optional unfreeze
    if cfg.freeze_backbone and epoch == cfg.unfreeze_epoch:
        print(f"[Info] Unfreezing image encoder at epoch {epoch}")
        set_freeze(img_encoder, False)
        optimizer = optim.AdamW([
            {"params": fusion_head.parameters(), "lr": cfg.lr_head},
            {"params": img_encoder.parameters(), "lr": cfg.lr_backbone},
        ], weight_decay=cfg.weight_decay)

    tr_loss, tr_acc, tr_f1 = run_epoch(train_loader, True, epoch, f"Epoch {epoch+1}/{cfg.epochs} [Train]")
    va_loss, va_acc, va_f1 = run_epoch(val_loader,   False, epoch, f"Epoch {epoch+1}/{cfg.epochs} [Val]")

    print(f"Epoch {epoch+1:02d} | "
          f"Train: loss {tr_loss:.4f} acc {tr_acc:.4f} f1 {tr_f1:.4f} || "
          f"Val: loss {va_loss:.4f} acc {va_acc:.4f} f1 {va_f1:.4f}")

    if va_acc > best_val_acc:
        best_val_acc = va_acc
        wait = 0
        torch.save({
            "img_encoder": img_encoder.state_dict(),
            "fusion_head": fusion_head.state_dict(),
            "classes": base_folder.classes,
            "feature_cols": feature_cols,
            "cfg": cfg.__dict__,
        }, "best_multimodal.pth")
        print(f"[+] Saved best model (val_acc={va_acc:.4f})")
    else:
        wait += 1
        if wait >= cfg.patience:
            print("⏹️ Early stopping.")
            break

# ----------------
# Test
# ----------------
ckpt = torch.load("best_multimodal.pth", map_location=cfg.device)
img_encoder.load_state_dict(ckpt["img_encoder"])
fusion_head.load_state_dict(ckpt["fusion_head"])
img_encoder.eval(); fusion_head.eval()

te_loss, te_acc, te_f1 = run_epoch(test_loader, False, 0, "Test")
print(f"TEST → Loss: {te_loss:.4f} | Acc: {te_acc:.4f} | F1: {te_f1:.4f}")


[Warn] /content/tabular_top10_features.csv not found. Falling back to /content/tabular_fixed.csv -> shape (2982, 9)
[Info] Using 6 tabular features: ['Age', 'Years_in_Education', 'APOE4', 'CSF_AB', 'CSF_P-tau181', 'CSF_T-tau']


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


[Info] Found 2982 images across 3 classes: ['AD', 'CN', 'MCI-1']
[train] Kept 2087 images | Missing tab: 0 | Class mismatch: 0
[eval] Kept 447 images | Missing tab: 0 | Class mismatch: 0
[eval] Kept 448 images | Missing tab: 0 | Class mismatch: 0


Epoch 01 | Train: loss 1.6924 acc 0.3589 f1 0.3513 || Val: loss 1.0995 acc 0.3602 f1 0.3515
[+] Saved best model (val_acc=0.3602)


Epoch 02 | Train: loss 1.2486 acc 0.3536 f1 0.3487 || Val: loss 1.1195 acc 0.2953 f1 0.2635


...

Epoch 47 | Train: loss 0.7854 acc 0.7034 f1 0.7016 || Val: loss 0.7346 acc 0.7069 f1 0.7023


Epoch 48 | Train: loss 0.7823 acc 0.7120 f1 0.7107 || Val: loss 0.7378 acc 0.7204 f1 0.7057


Epoch 49 | Train: loss 0.7640 acc 0.7230 f1 0.7226 || Val: loss 0.7099 acc 0.7248 f1 0.7091


Epoch 50 | Train: loss 0.7512 acc 0.7360 f1 0.7298 || Val: loss 0.7018 acc 0.7204 f1 0.7104
⏹️ Early stopping.


TEST → Loss: 0.7601 | Acc: 0.7210 | F1: 0.7107


In [10]:
# ================================
# Multimodal TransUNet + TabNet Top-K Fusion (No Early Stopping)
# ================================
# - Robust CSV ↔ image filename matching
# - Normalized class aliases
# - TransUNet encoder for images
# - TabNet (pytorch-tabnet) for tabular features selection (optional top-k CSV)
# - Concatenation fusion
# - Train for fixed epochs (no early stopping)

!pip install pytorch-tabnet -q

import os
import re
import random
from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score

# ----------------
# Config
# ----------------
class Cfg:
    data_root = "/content/dataset/Data/Data"
    tabular_csv = "/content/tabular_fixed.csv"
    topk_csv    = "/content/tabular_top10_features.csv"
    id_col = "ImageID"
    target_col = "Class"
    img_size = 224
    in_channels = 3
    batch_size = 16
    num_workers = 2
    seed = 42
    epochs = 100     # fixed training (no early stopping)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    lr_head = 1e-4
    lr_backbone = 1e-5
    weight_decay = 1e-4
    patch_size = 16
    embed_dim = 768
    num_heads = 8
    num_layers = 4
    freeze_backbone = True
    unfreeze_epoch = 5

cfg = Cfg()

def set_seed(s):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed(cfg.seed)

# ----------------
# Normalize helpers
# ----------------
def normalize_ext_to_jpg(name: str) -> str:
    n = os.path.basename(name.strip().lower())
    n = n.replace(".jpg.png", ".jpg").replace(".png.jpg", ".jpg")
    root, ext = os.path.splitext(n)
    if ext in [".jpeg", ".png", ".jpg"]:
        n = root + ".jpg"
    return n

def stem_without_ext(name: str) -> str:
    return os.path.splitext(os.path.basename(name.strip().lower()))[0]

def class_alias(name: str) -> str:
    if name is None: return ""
    s = str(name).strip()
    if s.upper() in ["MCI", "MCI-1"]: return "MCI-1"
    if s.upper() == "AD": return "AD"
    if s.upper() == "CN": return "CN"
    return s

# ----------------
# Load Tabular
# ----------------
if os.path.exists(cfg.topk_csv):
    df_tab = pd.read_csv(cfg.topk_csv)
    print(f"[Info] Loaded top-k features {df_tab.shape}")
else:
    df_tab = pd.read_csv(cfg.tabular_csv)
    print(f"[Warn] Using full tabular {df_tab.shape}")

drop_cols = {cfg.id_col, cfg.target_col}
if "Sex" in df_tab.columns: drop_cols.add("Sex")
feature_cols = [c for c in df_tab.columns if c not in drop_cols]

tab_dict: Dict[str, Tuple[np.ndarray, str]] = {}
for _, row in df_tab.iterrows():
    raw_id = str(row[cfg.id_col])
    feats = row[feature_cols].to_numpy(dtype=np.float32)
    raw_cls = class_alias(row[cfg.target_col])
    keys = {normalize_ext_to_jpg(raw_id), stem_without_ext(raw_id)}
    for k in keys:
        tab_dict[k] = (feats, raw_cls)

print(f"[Info] Features: {len(feature_cols)}")

# ----------------
# Dataset
# ----------------
def to_three_channels(img_tensor: torch.Tensor) -> torch.Tensor:
    return img_tensor.repeat(3,1,1) if img_tensor.ndim==3 and img_tensor.size(0)==1 else img_tensor

transform_train = transforms.Compose([
    transforms.Resize((cfg.img_size, cfg.img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(25),
    transforms.ToTensor(),
    transforms.Lambda(to_three_channels),
    transforms.Normalize([0.5]*3, [0.5]*3),
])
transform_eval = transforms.Compose([
    transforms.Resize((cfg.img_size, cfg.img_size)),
    transforms.ToTensor(),
    transforms.Lambda(to_three_channels),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

class MultiModalDataset(Dataset):
    def __init__(self, image_folder: ImageFolder, tab_dict, is_train=True):
        self.ifolder = image_folder
        self.tab_dict = tab_dict
        filtered_samples = []
        for path, y in self.ifolder.samples:
            base = os.path.basename(path)
            key = normalize_ext_to_jpg(base)
            key2 = stem_without_ext(base)
            row = tab_dict.get(key) or tab_dict.get(key2)
            if row:
                feats, csv_cls = row
                folder_cls = class_alias(self.ifolder.classes[y])
                if csv_cls == folder_cls:
                    filtered_samples.append((path, y))
        self.ifolder.samples = filtered_samples
        self.ifolder.targets = [s[1] for s in filtered_samples]

    def __len__(self): return len(self.ifolder)
    def __getitem__(self, idx):
        path, y = self.ifolder.samples[idx]
        img = self.ifolder.transform(Image.open(path).convert("RGB"))
        base = normalize_ext_to_jpg(os.path.basename(path))
        feats, _ = self.tab_dict.get(base) or self.tab_dict[stem_without_ext(base)]
        feats = torch.from_numpy(feats)
        return img, feats, y, Path(path).name

# ----------------
# Splits
# ----------------
base_folder = ImageFolder(cfg.data_root, transform=transform_train)
num_classes = len(base_folder.classes)
print(f"[Info] {len(base_folder)} images across {num_classes} classes")

# random split
all_idx = list(range(len(base_folder)))
random.shuffle(all_idx)
n = len(all_idx)
n_train, n_val = int(0.7*n), int(0.15*n)
train_idx, val_idx, test_idx = all_idx[:n_train], all_idx[n_train:n_train+n_val], all_idx[n_train+n_val:]

def subset(in_folder, idxs, transform):
    new = ImageFolder(in_folder.root, transform=transform)
    new.samples = [in_folder.samples[i] for i in idxs]
    new.targets = [s[1] for s in new.samples]
    return new

ds_train = MultiModalDataset(subset(base_folder, train_idx, transform_train), tab_dict, True)
ds_val   = MultiModalDataset(subset(base_folder, val_idx, transform_eval), tab_dict, False)
ds_test  = MultiModalDataset(subset(base_folder, test_idx, transform_eval), tab_dict, False)

train_loader = DataLoader(ds_train, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
val_loader   = DataLoader(ds_val,   batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)
test_loader  = DataLoader(ds_test,  batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

# ----------------
# Models
# ----------------
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, embed_dim, patch_size, patch_size)
        num_patches = (img_size//patch_size)**2
        self.cls = nn.Parameter(torch.zeros(1,1,embed_dim))
        self.pos = nn.Parameter(torch.zeros(1,num_patches+1,embed_dim))
    def forward(self,x):
        x = self.proj(x).flatten(2).transpose(1,2)
        cls = self.cls.expand(x.size(0),-1,-1)
        x = torch.cat([cls,x],1) + self.pos
        return x

class TransUNetEncoder(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, embed_dim, num_heads, num_layers):
        super().__init__()
        self.patch = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        enc_layer = nn.TransformerEncoderLayer(embed_dim, num_heads, embed_dim*4, 0.1, batch_first=False)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers)
        self.norm = nn.LayerNorm(embed_dim)
    def forward(self,x):
        x = self.patch(x).permute(1,0,2)
        x = self.encoder(x)[0]
        return self.norm(x)

class FusionHead(nn.Module):
    def __init__(self,img_dim,tab_dim,num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(img_dim+tab_dim,64),nn.ReLU(),nn.Dropout(0.3),
            nn.Linear(64,32),nn.ReLU(),nn.Dropout(0.3),
            nn.Linear(32,num_classes)
        )
    def forward(self,img_feat,tab_feat): return self.net(torch.cat([img_feat,tab_feat],1))

img_encoder = TransUNetEncoder(cfg.img_size,cfg.patch_size,cfg.in_channels,cfg.embed_dim,cfg.num_heads,cfg.num_layers).to(cfg.device)
fusion_head = FusionHead(cfg.embed_dim,len(feature_cols),num_classes).to(cfg.device)

# freeze/unfreeze
for p in img_encoder.parameters(): p.requires_grad = not cfg.freeze_backbone
optimizer = optim.AdamW([
    {"params": fusion_head.parameters(), "lr": cfg.lr_head},
    {"params": [p for p in img_encoder.parameters() if p.requires_grad], "lr": cfg.lr_backbone}
], weight_decay=cfg.weight_decay)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# ----------------
# Train (no early stopping)
# ----------------
def run_epoch(loader,train):
    img_encoder.train(train); fusion_head.train(train)
    total_loss,preds,labels=0,[],[]
    for imgs,tabs,ys,_ in loader:
        imgs,tabs,ys = imgs.to(cfg.device), tabs.to(cfg.device).float(), ys.to(cfg.device)
        with torch.set_grad_enabled(train):
            feats = img_encoder(imgs)
            out = fusion_head(feats,tabs)
            loss = criterion(out,ys)
            if train:
                optimizer.zero_grad(); loss.backward(); optimizer.step()
        total_loss += loss.item()*imgs.size(0)
        preds.extend(out.argmax(1).cpu().numpy()); labels.extend(ys.cpu().numpy())
    return total_loss/len(loader.dataset), accuracy_score(labels,preds), f1_score(labels,preds,average="macro")

for epoch in range(cfg.epochs):
    if cfg.freeze_backbone and epoch==cfg.unfreeze_epoch:
        for p in img_encoder.parameters(): p.requires_grad=True
        optimizer = optim.AdamW(list(img_encoder.parameters())+list(fusion_head.parameters()),lr=cfg.lr_backbone,weight_decay=cfg.weight_decay)
        print(f"[Info] Unfrozen backbone at epoch {epoch}")
    tr_loss,tr_acc,tr_f1 = run_epoch(train_loader,True)
    va_loss,va_acc,va_f1 = run_epoch(val_loader,False)
    print(f"Epoch {epoch+1:02d}/{cfg.epochs} | Train {tr_loss:.4f} acc {tr_acc:.4f} f1 {tr_f1:.4f} || Val {va_loss:.4f} acc {va_acc:.4f} f1 {va_f1:.4f}")

# ----------------
# Test
# ----------------
te_loss,te_acc,te_f1 = run_epoch(test_loader,False)
print(f"TEST → Loss {te_loss:.4f} | Acc {te_acc:.4f} | F1 {te_f1:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.6 MB/s eta 0:00:00
[Warn] Using full tabular (2982, 9)
[Info] Features: 6
[Info] 2982 images across 3 classes


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 01/100 | Train 1.6568 acc 0.3594 f1 0.3513 || Val 1.0994 acc 0.3445 f1 0.3025
Epoch 02/100 | Train 1.2583 acc 0.3507 f1 0.3449 || Val 1.1043 acc 0.3311 f1 0.3198
Epoch 03/100 | Train 1.1663 acc 0.3349 f1 0.3265 || Val 1.1103 acc 0.3132 f1 0.2831
Epoch 04/100 | Train 1.1377 acc 0.3373 f1 0.3337 || Val 1.1004 acc 0.3445 f1 0.3446
Epoch 05/100 | Train 1.1242 acc 0.3402 f1 0.3272 || Val 1.1029 acc 0.3423 f1 0.2896
[Info] Unfrozen backbone at epoch 5
Epoch 06/100 | Train 1.1184 acc 0.3445 f1 0.3220 || Val 1.1021 acc 0.3423 f1 0.2952
Epoch 07/100 | Train 1.1074 acc 0.3637 f1 0.3447 || Val 1.0980 acc 0.3468 f1 0.3106
Epoch 08/100 | Train 1.0986 acc 0.3805 f1 0.3573 || Val 1.0674 acc 0.4653 f1 0.3686
Epoch 09/100 | Train 1.0636 acc 0.4327 f1 0.4047 || Val 1.0069 acc 0.4989 f1 0.4930
Epoch 10/100 | Train 1.0312 acc 0.4705 f1 0.4554 || Val 0.9939 acc 0.5145 f1 0.4951
Epoch 11/100 | Train 1.0207 acc 0.4777 f1 0.4678 || Val 0.9980 acc 0.5168 f1 0.4802
Epoch 12/100 | Train 1.0135 acc 0.4921 f